In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.61  # https://github.com/abetlen/llama-cpp-python, CUBLAS is for GPU usage
!pip -q install  transformers langchain

In [3]:
%%capture
!wget -O model.gguf https://huggingface.co/TheBloke/Starling-LM-7B-alpha-GGUF/resolve/main/starling-lm-7b-alpha.Q5_K_M.gguf  # сама gguf-модель, скачиваем с huggingface

In [4]:
from langchain.prompts import PromptTemplate
from llama_cpp import Llama
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F


In [ ]:
llm = Llama(
  model_path="model.gguf",
  n_ctx=8192,  # длина контекста, воспринимаемого моделью # можно менять
  n_threads=8,
  n_gpu_layers=-1  # сколько слоёв GPU сгружать, -1 значит все. Starling влезает весь, чего не скажешь, например, о Mixtral
)


In [6]:
# гиперпараметры для генерации
llm_kwargs = {
    'max_tokens': 2048,
    'stop': ["<|end_of_turn|>"],
    'temperature': 1.5,
    'top_p': 1
}

In [ ]:
sent_tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
sent_model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

In [8]:
def generate_prompt(question, text):
    return f"""GPT4 Correct User: {question}
    <text>{text}</text><|end_of_turn|>
    GPT4 Correct Assistant:"""

def generate_answer(prompt):
    global llm
    with torch.no_grad():
        output = llm(prompt, **llm_kwargs)['choices'][0]['text'].strip()
    return output

def generate_answer_with_chunking(question, file_path, chunk_size=512):
    answers = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for chunk in read_in_chunks(file, chunk_size):
            chunk = chunk.replace("{", "{{").replace("}", "}}")  # Escape characters right before use
            message = generate_prompt(question, chunk)
            with torch.no_grad():
                answer = generate_answer(message, verbose=False, **llm_kwargs)
            answers.append(answer)
    return " ".join(answers)


In [9]:
def get_embedding(sentence):
    encoded_input = sent_tokenizer([sentence], padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = sent_model(**encoded_input)
    token_embeddings = model_output[0]
    input_mask_expanded = encoded_input['attention_mask'].unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [10]:
def read_in_chunks(file_object, chunk_size=512):
    while True:
        data = file_object.read(chunk_size)
        if not data:
            break
        yield data

In [17]:
# Функция для вычисления эмбеддингов файла
def get_embeddings_from_file(file_path):
    embeddings = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for chunk in read_in_chunks(f):
            if not chunk.strip():  # Игнорируем пустые части
                continue
            chunk_embeddings = get_embedding(chunk)
            embeddings.append(chunk_embeddings)
    if embeddings:
        final_embedding = torch.mean(torch.stack(embeddings), dim=0)
        print(f"Embedding for {file_path} calculated successfully.")
        return final_embedding
    else:
        print(f"No valid embeddings found for {file_path}.")
        return torch.zeros_like()


In [18]:
def find_most_relevant_file(question_embedding):
    max_similarity = -1  # Инициализация максимального сходства как очень маленькое значение
    relevant_file_path = None  # Инициализация пути к наиболее релевантному файлу

    # Проход по всем файлам и их эмбеддингам
    for file_path, embedding in file_embeddings.items():
        if embedding.nelement() == 0:
            continue  # Пропускаем файлы с нулевыми эмбеддингами

        # Убедимся, что эмбеддинги имеют размерность (1, D), где D - размер эмбеддинга
        embedding = embedding.unsqueeze(0) if embedding.dim() == 1 else embedding
        question_embedding = question_embedding.unsqueeze(0) if question_embedding.dim() == 1 else question_embedding

        # Вычисление косинусного сходства между эмбеддингом файла и эмбеддингом вопроса
        similarity = F.cosine_similarity(embedding, question_embedding, dim=1)

        # Проверяем, что размерность similarity == (1,)
        if similarity.size(0) != 1:
            raise ValueError(f"Expected similarity to be a scalar, got {similarity.size()}")

        similarity_value = similarity.item()

        print(f"File: {file_path}, Similarity: {similarity_value}")  # Для отладки
        if similarity_value > max_similarity:
            max_similarity = similarity_value
            relevant_file_path = file_path

    return relevant_file_path


In [ ]:
# add your document file path
file_paths = [ 'txt1.txt', 'txt2.txt', 'txt3.txt', 'txt4.txt']
file_embeddings = {path: get_embeddings_from_file(path) for path in file_paths}

In [20]:
def generate_answer_for_question(question):
    question_embedding = get_embedding(question)
    relevant_file_path = find_most_relevant_file(question_embedding)
    if relevant_file_path:
        with open(relevant_file_path, 'r', encoding='utf-8') as file:
            full_text = file.read()
        prompt = generate_prompt(question, full_text)
        answer = generate_answer(prompt)
        return answer, relevant_file_path
    else:
        return "No relevant information found.", None


In [21]:
questions = ["Перечисли основные виды ресурсов, такие как Информация, Организованность и тд и дай им определения?","Что такое система? Базовое определение", "Напиши определение системы", "Что такое «Цель»?"]

In [ ]:
for question in questions:
    answer, file_path = generate_answer_for_question(question)
    print(f"Вопрос: {question}")
    if file_path:
        print(f"Ответ в файле: {file_path}")
    print("ответ:", answer)
    print()